In [25]:
import os
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.model_selection import StratifiedShuffleSplit, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVR, SVR
from source.DataFrameSelector import DataFrameSelector

DATA_PATH = os.path.join("..", "data")

def load_csv(filename, path=DATA_PATH):
    return pd.read_csv(os.path.join(path, filename))

In [3]:
def import_data():
    e_data = load_csv("energy_dataset.csv")
    w_data = load_csv("weather_features.csv")
    # weather_id ist abgeleitet von clouds_all
    # weather_id ist die numerische klassifikation von weather_description
    # weather_main ist eine ungenaue darstellung von weather_description
    # weather_icon ist ?
    # in den meisten fällen ist temp = temp_min = temp_max
    w_data = w_data.drop(columns=["weather_id", "weather_icon", "weather_main", "temp_min", "temp_max"])
    return w_data.join(other=e_data[["time", "price"]].set_index("time"), on="time")

data = import_data()

In [23]:
dfs = [data.loc[data["city_name"] == city] for city in data["city_name"].value_counts().index]

5
<class 'pandas.core.frame.DataFrame'>
Int64Index: 35951 entries, 71412 to 107362
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   time                 35951 non-null  object 
 1   city_name            35951 non-null  object 
 2   temp                 35951 non-null  float64
 3   pressure             35951 non-null  int64  
 4   humidity             35951 non-null  int64  
 5   wind_speed           35951 non-null  int64  
 6   wind_deg             35951 non-null  int64  
 7   rain_1h              35951 non-null  float64
 8   rain_3h              35951 non-null  float64
 9   snow_3h              35951 non-null  float64
 10  clouds_all           35951 non-null  int64  
 11  weather_description  35951 non-null  object 
 12  price                35951 non-null  float64
dtypes: float64(5), int64(5), object(3)
memory usage: 3.8+ MB


In [69]:
label = "price"
sets = []
for df in dfs:
    X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=[label]), df[label], test_size=0.2, random_state=42)
    sets.append({"X_train": X_train, "X_test": X_test, "y_train": y_train, "y_test": y_test})

{'X_train':                             time city_name     temp  pressure  humidity  \
 12788  2016-06-15 14:00:00+02:00  Valencia  300.920      1007        23   
 23213  2017-08-22 20:00:00+02:00  Valencia  302.150      1015        54   
 23006  2017-08-14 05:00:00+02:00  Valencia  295.150      1018        78   
 18955  2017-02-26 19:00:00+01:00  Valencia  287.150      1019        76   
 17497  2016-12-28 03:00:00+01:00  Valencia  279.150      1035       100   
 ...                          ...       ...      ...       ...       ...   
 16850  2016-12-01 04:00:00+01:00  Valencia  286.150      1029        82   
 6265   2015-09-18 15:00:00+02:00  Valencia  296.590      1020        41   
 11284  2016-04-14 09:00:00+02:00  Valencia  290.300      1016        62   
 860    2015-02-05 19:00:00+01:00  Valencia  279.350      1010        34   
 15795  2016-10-18 20:00:00+02:00  Valencia  291.802       997        65   
 
        wind_speed  wind_deg  rain_1h  rain_3h  snow_3h  clouds_all  \
 127

In [36]:
cat_attribs = ["weather_description"]
num_attribs = ['rain_1h', 'rain_3h', 'snow_3h', 'clouds_all']
poly_attribs = ['temp', 'pressure', 'humidity', 'wind_speed', 'wind_deg']
degree = 2

cat_pipeline = Pipeline([
    ("selector", DataFrameSelector(cat_attribs)),
    ("cat_encoder", OneHotEncoder(sparse=False))
])

pipelines = [FeatureUnion(transformer_list=[
    ("num_pipeline", Pipeline([
        ("selector", DataFrameSelector(num_attribs)),
        ("std_scaler", StandardScaler())
    ])),
    ("poly_pipeline", Pipeline([
        ("selector", DataFrameSelector(poly_attribs)),
        ("poly_features", PolynomialFeatures(degree=degree)),
        ("std_scaler", StandardScaler())
    ])),
]).fit(_set["X_train"]) for _set in sets]

for _set, pipeline in zip(sets, pipelines):
    _set["X_train_clean"] = pipeline.transform(_set["X_train"])
    _set["X_test_clean"] = pipeline.transform(_set["X_test"])

In [62]:
models = [LinearRegression().fit(_set["X_train_clean"], _set["y_train"]) for _set, pipeline in zip(sets, pipelines)]
predictions = []
predictions_test = []
for count, (model, _set) in enumerate(zip(models, sets)):
    rmse_train = np.sqrt(mean_squared_error(_set["y_train"], model.predict(_set["X_train_clean"])))
    rmse_test = np.sqrt(mean_squared_error(_set["y_test"], model.predict(_set["X_test_clean"])))
    print(rmse_train, rmse_test)
    predictions.append(model.predict(_set["X_train_clean"]))
    predictions_test.append(model.predict(_set["X_test_clean"]))

predictions.append(min(sets, key = lambda x: x["y_train"].__len__())["y_train"])
predictions_test.append(min(sets, key = lambda x: x["y_test"].__len__())["y_test"])
foo = pd.DataFrame(predictions).transpose().dropna()
test = pd.DataFrame(predictions_test).transpose().dropna()    

13.445103686395514 13.336254276202173
13.378419901654263 13.283128586884326
13.680226403441372 13.798161356271763
13.705752353752203 13.64433289339329
13.532891279795942 13.407035614843604


In [70]:
#foo.drop(columns=[5]).info()
lin_reg = LinearRegression()
lin_reg.fit(foo.drop(columns=[5]), foo[5])

rmse_train = np.sqrt(mean_squared_error(foo[5], lin_reg.predict(foo.drop(columns=[5]))))
rmse_test = np.sqrt(mean_squared_error(test[5], lin_reg.predict(test.drop(columns=[5]))))
print(rmse_train, rmse_test)
print(foo)

13.532860596691599 13.407161181131517
               0          1          2          3          4      5
0      56.374478  61.809020  57.800531  51.318967  62.646476  41.00
1      52.561970  56.500003  58.187785  59.150107  62.639794  58.88
2      61.780262  58.219770  52.102411  60.365653  60.194456  47.04
3      59.202358  50.341749  54.351494  57.369213  56.573405  61.78
4      61.522934  59.636782  60.212418  61.615103  63.231450  51.05
...          ...        ...        ...        ...        ...    ...
28111  56.686472  55.936363  60.438010  56.013193  60.332211  55.31
28112  60.595679  61.536681  58.517505  54.129953  57.502875  58.46
28113  50.908389  61.003802  63.378334  60.223643  55.448026  36.83
28114  58.359588  63.231506  55.170224  59.703563  46.694973  82.79
28115  58.884888  54.676976  62.309788  59.292679  58.775481  74.68

[28116 rows x 6 columns]
